In [31]:
import pandas as pd
import os
import requests
import json
from dotenv import load_dotenv

Variables List + Header API

In [30]:
load_dotenv()
RIOT_API_KEY = os.getenv("RIOT_API_KEY")
my_game_name = os.getenv("my_game_name")
my_tag_line = os.getenv("my_tag_line")

In [32]:
headers = {
    "X-Riot-Token" : RIOT_API_KEY
}

# Creating a quick data frame just for the purpose of testing the functions in the blocks below

In [24]:
api_url = "https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"

resp = requests.get(api_url, headers=headers)
response = resp.json()
entries = response.get('entries', [])
entries = entries[:4]
test_df = pd.DataFrame([{"summonerId" : entry["summonerId"]} for entry in entries])
test_df.to_csv("test.csv", index = False)

# Function to read csv above and populate it (and the dataframe) with puuids in a new column

In [ ]:
def get_puuid(csv_only_summ_ids):
    # Assigning csv file here as a placeholder, can just include it in the argument when using this fucntion
    df_only_summ_ids = pd.read_csv(csv_only_summ_ids, index_col=False)
    # Loop to iterate through each row
    for index, row in df_only_summ_ids.iterrows():
            summoner_id = row["summonerId"]
            # Variable to insert summ_id in API url that fetches account data
            api_url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
            try:
                resp = requests.get(api_url, headers=headers)  #Insert API link here
                resp.raise_for_status()
                summoner_info = resp.json()
                # Create a column in the data frame called puuids and populate it with the puuid for each summoner id
                df_only_summ_ids.at[index, "puuid"] = summoner_info.get("puuid", "Not Found") # Store puuid in df, output Not Found if KeyError
            # Alternative way to handle request exceptions that handles all request Errors instead of just error 400
            except requests.RequestException as e:
                print(f"Error fetching puuid for {summoner_id}: {e}")
    # Update csv file with the new column
    df_only_summ_ids.to_csv(csv_only_summ_ids, index = False)
# Calling function to add new info to existing csv instead of creating new one (using test csv from above in this case)
get_puuid("test.csv")

# Function to get match ids using puuids from Dataframe

In [ ]:
def get_match_ids(csv_summids_and_puuids):
    # Read csv file with summoner ids and puuids 
    df_summoners = pd.read_csv(csv_summids_and_puuids)
    # Create a new DataFrame to store match data
    df_all_matches = pd.DataFrame(columns = ["summonerId", "puuid", "matchId", "matchData"])
    # Variable for number of matches, will be defaulted to max (100) in actual code, small number used for testing
    number_of_matches = 5
    # Iterate through each row in df_summoners
    for _, row in df_summoners.iterrows():
        summoner_id = row["summonerId"]
        puuid = row["puuid"]
        # Fetch match ids for each puuid
        match_history_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={number_of_matches}"

        try:
            resp = requests.get(match_history_api_url, headers=headers)
            resp.raise_for_status()
            match_ids = resp.json()
            # Iterate through each match id and fetch match data
            for match_id in match_ids:
                match_data_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
                response = requests.get(match_data_api_url, headers=headers)
                match_data = response.json()
                # Store match data in JSON file along with summoner id and puuid
                df_mini = pd.DataFrame({
                    "summonerId" : [summoner_id],
                    "puuid" : [puuid],
                    "matchId" : [match_id],
                    "matchData": [json.dumps(match_data)]
                    })
                # Append to main DataFrame
                df_all_matches = pd.concat([df_all_matches, df_mini], ignore_index = True)
        # Alternative way to handle request exceptions that handles all request Errors instead of just error 400
        except requests.RequestException as e:
            print(f"Error fetching matches for {puuid}: {e}")
    # Save the expanded DataFrame
    df_all_matches.to_csv("all_match_data_test.csv", index = False)
get_match_ids("test.csv")